In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from PIL import Image

# Filter harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# the internal service endpoint must be completed with the model inference point in the format /v2/models/<model_name>/infer
ml_endpoint = 'http://modelmesh-serving.test:8008/v2/models/xraydemo/infer'

In [4]:
headers = {
    'content-type': 'application/json'
} 

In [5]:
test_transform = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225]
#        mean = [0.5, 0.5, 0.5],
#        std = [0.5, 0.5, 0.5]
    )
])

In [6]:
file_name = './demo_Adam Lomboy_2523_1966-08-19_2016-03-18.jpeg'

In [7]:
img = Image.open(file_name)

In [8]:
# we need 3 channels for the 
img_conv=img.convert(mode="RGB")

In [9]:
img_tensor_conv = test_transform(img_conv)

In [10]:
# expand the tensor with one more dimension that stands in for the batch
img_tensor_conv = torch.unsqueeze(img_tensor_conv, 0)

In [11]:
img_tensor_conv.shape

torch.Size([1, 3, 150, 150])

In [12]:
img_array = img_tensor_conv.tolist()

In [13]:
message = {
    "inputs":[
        {
            "name":"input.1",
            "shape":[
                1,
                3,
                150,
                150
            ],
            "datatype":"FP32",
            "data": img_array
        }
    ]        
}

In [14]:
import requests

In [15]:
resp = requests.post(url=ml_endpoint, json=message, headers=headers)

In [16]:
resp.json()

{'model_name': 'xraydemo__isvc-55544ff705',
 'model_version': '1',
 'outputs': [{'name': '25',
   'datatype': 'FP32',
   'shape': [1, 2],
   'data': [-7.273074, 6.676987]}]}

In [17]:
# reshape the output data to a 1,2 numpy array
pred = np.array(resp.json()['outputs'][0]['data']).reshape(1,-1)

In [18]:
# convert to torch tensor and get the indices of the prediction class
t_pred = torch.from_numpy(pred)
predicted = torch.max(t_pred.data, 1)[1]

In [20]:
classes = ['NORMAL', 'PNEUMONIA']

In [21]:
# display the prediction class
classes[predicted.item()]

'PNEUMONIA'